<a href="https://colab.research.google.com/github/HarryWightJr/final_project/blob/main/linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graph

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-13 20:53:50--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.95MB/s    in 0.2s    

2022-03-13 20:53:51 (4.95 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://bootcampfinalproject.s3.amazonaws.com/Resources/complete_df.csv"
spark.sparkContext.addFile(url)
complete_df = spark.read.csv(SparkFiles.get("complete_df.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
complete_df.show()

+---+--------+---------------------+---------------------+---------------+------------------------+-------------------------+---------------+------------------------+-------------------------+-----------+-----------+
|_c0|CountyID|AverageSalesPrice2019|AverageSalesPrice2020|Population 2019|Violent Crime Total 2019|Property Crime Total 2019|Population 2020|Violent Crime Total 2020|Property Crime Total 2020|Income_2019|Income_2020|
+---+--------+---------------------+---------------------+---------------+------------------------+-------------------------+---------------+------------------------+-------------------------+-----------+-----------+
|  0| 26017.0|              59150.0|              64300.0|        32793.0|                   271.0|                    654.0|        32485.0|                   276.0|                    500.0|    43554.0|    47786.0|
|  4| 19179.0|              77258.0|              68800.0|        24488.0|                   134.0|                   1009.0|       

In [5]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.linear_model import LinearRegression
import math
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [6]:
complete_df.dtypes

[('_c0', 'int'),
 ('CountyID', 'double'),
 ('AverageSalesPrice2019', 'double'),
 ('AverageSalesPrice2020', 'double'),
 ('Population 2019', 'double'),
 ('Violent Crime Total 2019', 'double'),
 ('Property Crime Total 2019', 'double'),
 ('Population 2020', 'double'),
 ('Violent Crime Total 2020', 'double'),
 ('Property Crime Total 2020', 'double'),
 ('Income_2019', 'double'),
 ('Income_2020', 'double')]

In [7]:
X = np.array(complete_df[['CountyID','AverageSalesPrice2019','Population 2019','Violent Crime Total 2019','Property Crime Total 2019','Population 2020','Violent Crime Total 2020','Property Crime Total 2020','Income_2019','Income_2020']])
Y = np.array(complete_df['AverageSalesPrice2020'])

In [8]:
model = LinearRegression().fit(X, Y)

ValueError: ignored